In [3]:
import requests
import pandas as pd

# Get Metadata

We can use the autocomplete API to get the list of available tables and fields to build our CSV exports.

In [17]:
autocomplete_url = "https://dragalialost.gamepedia.com/api.php?action=cargoqueryautocomplete&format=json&search="
result_key = "cargoqueryautocomplete"

## Tables

In [25]:
tables = requests.get(autocomplete_url).json()[result_key]
pd.DataFrame(tables)

,main_table
0,ABRCharacter
1,ABRDragon
2,Abilities
3,AbilityGroup
4,AbilityLimitedGroup
...,...
56,WyrmprintRarity
57,WyrmprintTrade
58,Wyrmprints
59,Wyrmprintsv1


## Fields

In [51]:
def get_fields(table):
    table_name = table["main_table"]
    fields = requests.get(autocomplete_url, params={"tables": table_name}).json()[result_key]
    return {**table, **{"fields": fields}}

In [60]:
import concurrent.futures
import time

start = time.time()

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(get_fields, table) for table in tables]
    tables_and_fields = [f.result() for f in futures]
    
print(f"Elapsed Time:{time.time() - start}")

Elapsed Time:0.8138711452484131


In [61]:
pd.DataFrame(tables_and_fields)

,main_table,fields
0,ABRCharacter,"[ABRCharacter.Id, ABRCharacter.Portrait, ABRCh..."
1,ABRDragon,"[ABRDragon.Id, ABRDragon.Portrait, ABRDragon.N..."
2,Abilities,"[Abilities.Id, Abilities.GenericName, Abilitie..."
3,AbilityGroup,"[AbilityGroup.Id, AbilityGroup.GroupName, Abil..."
4,AbilityLimitedGroup,"[AbilityLimitedGroup.Id, AbilityLimitedGroup.I..."
...,...,...
56,WyrmprintRarity,"[WyrmprintRarity.Id, WyrmprintRarity.MaxLimitL..."
57,WyrmprintTrade,"[WyrmprintTrade.Id, WyrmprintTrade.AbilityCres..."
58,Wyrmprints,"[Wyrmprints.Id, Wyrmprints.BaseId, Wyrmprints...."
59,Wyrmprintsv1,"[Wyrmprintsv1.Id, Wyrmprintsv1.BaseId, Wyrmpri..."


# Get CSVs

Now that we have the metadata, let's just grab all the data so we can mess around with it.